<a href="https://colab.research.google.com/github/junkyuhufs/2024WorkshopForProfs/blob/main/Writing_checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install language_tool_python gradio

import gradio as gr
import language_tool_python

# Initialize the LanguageTool object for English
tool = language_tool_python.LanguageTool('en-US')

# Grammar and style check function
def check_grammar(text):
    matches = tool.check(text)
    if not matches:
        return "No mistakes found!", "Great job! Your text is perfect!"
    else:
        # Apply corrections
        corrected_text = text
        for match in reversed(matches):  # Reverse to avoid index shifting
            start, end = match.offset, match.offset + match.errorLength
            corrected_text = corrected_text[:start] + match.replacements[0] + corrected_text[end:]

        # Create detailed feedback
        feedback = "\n".join(
            f"Error {i + 1}: '{match.context}'\nSuggestion: {', '.join(match.replacements) or 'No suggestions available'}\nMessage: {match.message}\n"
            for i, match in enumerate(matches)
        )
        return corrected_text, f"Found {len(matches)} mistakes:\n\n{feedback}"

# Gradio interface
with gr.Blocks() as iface:
    with gr.Row():
        gr.Markdown("# Gradio Writing Checker App")

    gr.Markdown(
        """
        ### ✨ Improve Your Writing Instantly!
        This app analyzes your text for grammar and style issues. It will:
        - Highlight mistakes
        - Suggest corrections
        - Explain the issues found

        Just enter your text below and click "Check Grammar"!

        ---

        ### Writing Prompt:
        In the story, the Fox tries repeatedly to reach a tempting bunch of grapes, only to give up and convince himself that the grapes were not worth the effort.

        Have you ever had an experience where you worked hard to achieve something but decided to give up in the end? Share your story.
        - Describe what you were trying to accomplish, why you stopped, and how you felt afterward.
        - Reflect on your decision and relate it to the lesson in the story.
        - Remember to use past tense in your writing and provide reasons for your feelings and actions.
        """
    )

    with gr.Row():
        # User input text box
        input_text = gr.Textbox(label="Enter your text here", placeholder="Type your text...", lines=6)

    with gr.Row():
        # Output sections
        corrected_output = gr.Textbox(label="Corrected Text", placeholder="Your corrected text will appear here.", lines=6)
        feedback_output = gr.Textbox(label="Detailed Feedback", placeholder="Error explanations will appear here.", lines=10)

    # Grammar check button
    btn = gr.Button("Check Grammar")

    # Button click triggers the grammar check
    btn.click(fn=check_grammar, inputs=input_text, outputs=[corrected_output, feedback_output])

iface.launch()